In [1]:
from datetime import datetime
from os.path import join
from os import listdir
import json
import re #for camel case conversion
from collections import Counter

from sklearn.metrics import classification_report
import pandas as pd

import altair as alt
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [2]:
def sherlock_case(s):
    s = re.sub(r"(_|-)+", " ", s).title().replace(" ", "")
#     s = ''.join([s[0].lower(), s[1:]])
#     s = ''.join(map(lambda x: x if x.islower() else " "+x, s))
    return s.lower()

In [3]:
true_types = list(map(sherlock_case, pd.read_parquet("../../results/true_types/gittables_benchmark.parquet").values.flatten()))
prediction_sherlock = list(map(sherlock_case, pd.read_parquet("../../results/predictions/sherlock_gittables_benchmark.parquet").values.flatten()))
prediction_sato = list(map(sherlock_case, pd.read_parquet("../../results/predictions/sato_gittables_benchmark.parquet").values.flatten()))

In [4]:
type_freq_df = pd.DataFrame(true_types, columns=['type'])
type_freq_df = pd.DataFrame(type_freq_df['type'].value_counts())
type_freq_df.columns = ['count']
type_freq_df.index.name = 'type'

In [5]:
alt.Chart(type_freq_df.reset_index()).mark_bar(size=15).encode(
    x = alt.X('type:O',
              title = 'Semantic Types',
              sort=alt.EncodingSortField(
                field="count",  
                order="descending")),
    y = alt.Y('count', title='Number of Samples')    
)

alt.Chart(...)

In [6]:
print(len(prediction_sherlock))
print(len(prediction_sato))
print(len(true_types))


801
801
801


In [7]:
# print(prediction_sato)
# print(prediction_sherlock)
# print(true_types)
# for idx, i in enumerate(true_types):
#     if type(i) != str:
#         print(idx)
# print(true_types[407])
# print(prediction_sherlock[407])

In [8]:
# print(classification_report(true_types, prediction_sato))

In [9]:
# class_report = classification_report(true_types, prediction_sato, output_dict=True)
# class_report = dict(list(class_report.items()))
# class_report_df = pd.DataFrame.from_dict(class_report)
# class_report_df.to_csv('csv_report_sato_benchmark.csv')

In [10]:
# print(classification_report(true_types, prediction_sherlock))

In [11]:
# class_report = classification_report(true_types, prediction_sherlock, output_dict=True)
# class_report = dict(list(class_report.items()))
# class_report_df = pd.DataFrame.from_dict(class_report)
# class_report_df.to_csv('csv_report_sherlock_benchmark.csv')

In [12]:
report_sherlock = classification_report(true_types, prediction_sherlock, output_dict=True)
report_sherlock_df_input = {k: list(v.values()) for k, v in list(report_sherlock.items())[:-3]} #last 3 are total f1/macro/weigthed, these are not needed
report_sherlock_df = pd.DataFrame.from_dict(report_sherlock_df_input, orient='index', columns=['precision', 'recall', 'f1-score', 'support']).sort_values(by='f1-score', ascending = False)

report_sato = classification_report(true_types, prediction_sato, output_dict=True)
report_sato_df_input = {k: list(v.values()) for k, v in list(report_sato.items())[:-3]} #last 3 are total f1/macro/weigthed, these are not needed
report_sato_df = pd.DataFrame.from_dict(report_sato_df_input, orient='index', columns=['precision', 'recall', 'f1-score', 'support']).sort_values(by='f1-score', ascending = False)


/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/virtualenvs/sherlock/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/senn/v

In [13]:
combined_report_df = pd.merge(report_sato_df, report_sherlock_df, left_index=True, right_index=True)
combined_report_df.index.names = ['type']
combined_report_df.columns = ['precision_sato', 'recall_sato', 'f1-score_sato', 'support_sato', 'precision_sherlock', 'recall_sherlock', 'f1-score_sherlock', 'support_sherlock']

In [14]:
combined_report_df

,precision_sato,recall_sato,f1-score_sato,support_sato,precision_sherlock,recall_sherlock,f1-score_sherlock,support_sherlock
type,,,,,,,,
address,1.000000,1.000000,1.000000,1,0.007194,1.000000,0.014286,1
gender,1.000000,1.000000,1.000000,1,0.000000,0.000000,0.000000,1
year,0.980769,1.000000,0.990291,102,1.000000,0.911765,0.953846,102
city,0.714286,1.000000,0.833333,5,0.833333,1.000000,0.909091,5
country,0.750000,0.750000,0.750000,4,0.750000,0.750000,0.750000,4
name,0.676471,0.663462,0.669903,208,0.800000,0.076923,0.140351,208
description,0.470588,0.727273,0.571429,33,0.413793,0.363636,0.387097,33
status,0.347826,0.571429,0.432432,14,0.588235,0.714286,0.645161,14
code,0.714286,0.294118,0.416667,17,0.800000,0.235294,0.363636,17


In [15]:
mismatches_sherlock = list()
mismatches_sherlock_idx = list()
mismatches_sato = list()
mismatches_sato_idx = list()

print_count = 0
for idx, true_type in enumerate(true_types):
    predicted_type_sherlock = prediction_sherlock[idx]
    predicted_type_sato = prediction_sato[idx]

    if true_type != predicted_type_sherlock:
        mismatches_sherlock.append(true_type)
        mismatches_sherlock_idx.append(idx)

    if true_type != predicted_type_sato:
        mismatches_sato.append(true_type)
        mismatches_sato_idx.append(idx)
        
        # zoom in to specific errors
        if true_type in ('state') and print_count <= 6:
            print_count += 1
            print(f'Sherlock, Expected "{true_type}" but predicted "{predicted_type_sherlock}"')
            print(f'Sato, Expected "{true_type}" but predicted "{predicted_type_sato}"')
            
#             print(f'{data[idx]}\n')
        

mismatch_sherlock_class_count = Counter(mismatches_sherlock)
print(mismatch_sherlock_class_count.most_common()[:10])

mismatches_sato_class_count = Counter(mismatches_sato)
print(mismatches_sato_class_count.most_common()[:10])

Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "type"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
Sherlock, Expected "state" but predicted "category"
Sato, Expected "state" but predicted "type"
[('name', 192), ('rank', 97), ('class', 60), ('type', 54), ('description', 21), ('state', 19), ('code', 13), ('species', 13), ('depth', 13), ('duration', 11)]
[('type', 114), ('rank', 100), ('name', 70), ('class', 63), ('species', 26), ('state', 18), ('depth', 13), ('code', 12), ('duration', 11), ('description', 9)]


In [16]:
mismatch_sherlock_freq_df = pd.DataFrame(mismatches_sherlock, columns=['type'])
mismatch_sherlock_freq_df = pd.DataFrame(mismatch_sherlock_freq_df['type'].value_counts())
mismatch_sherlock_freq_df.columns = ['count']
mismatch_sherlock_freq_df.index.name = 'type'

mismatch_sato_freq_df = pd.DataFrame(mismatches_sato, columns=['type'])
mismatch_sato_freq_df = pd.DataFrame(mismatch_sato_freq_df['type'].value_counts())
mismatch_sato_freq_df.columns = ['count']
mismatch_sato_freq_df.index.name = 'type'

type_freq_df = pd.DataFrame(true_types, columns=['type'])
type_freq_df = pd.DataFrame(type_freq_df['type'].value_counts())
type_freq_df.columns = ['count']
type_freq_df.index.name = 'type'

In [17]:
combined_mismatch_freq_df = pd.merge(mismatch_sherlock_freq_df, mismatch_sato_freq_df, left_index=True, right_index=True, how='outer')
combined_mismatch_freq_df = pd.merge(combined_mismatch_freq_df, type_freq_df, left_index=True, right_index=True, how='outer')
combined_mismatch_freq_df.columns=['sherlock_mismatch_freq', 'sato_mismatch_freq', 'true_type_freq']

In [18]:
alt.Chart(combined_mismatch_freq_df.reset_index()).transform_fold(
      ['sherlock_mismatch_freq', 'sato_mismatch_freq', 'true_type_freq'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="sato_mismatch_freq",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [19]:
# print(combined_mismatch_freq_df)

In [20]:
alt.Chart(combined_report_df.reset_index()).transform_fold(
      ['precision_sato', 'recall_sato', 'f1-score_sato', 'precision_sherlock', 'recall_sherlock', 'f1-score_sherlock'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="support_sherlock",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [21]:
score_improvement_df = pd.DataFrame()
#precision, recall and f1-score
score_improvement_df["precision"] = report_sato_df["precision"] - report_sherlock_df["precision"]
score_improvement_df["recall"] = report_sato_df["recall"] - report_sherlock_df["recall"]
score_improvement_df["f1-score"] = report_sato_df["f1-score"] - report_sherlock_df["f1-score"]
score_improvement_df["support"] = report_sato_df["support"]
score_improvement_df.index.name = 'type'

In [22]:
score_improvement_df

,precision,recall,f1-score,support
type,,,,
address,0.992806,0.000000,0.985714,1.0
affiliation,NaN,NaN,NaN,0.0
age,0.000000,0.000000,0.000000,1.0
album,0.000000,0.000000,0.000000,0.0
area,0.000000,0.000000,0.000000,0.0
...,...,...,...,...
team,0.000000,0.000000,0.000000,2.0
teamname,NaN,NaN,NaN,0.0
type,-0.545926,-0.425532,-0.484952,141.0


In [23]:
alt.Chart(score_improvement_df.reset_index()).transform_fold(
      ['precision', 'recall', 'f1-score'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="f1-score",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [24]:
sato_top_5_df = report_sato_df.loc[report_sato_df['support']>0].sort_values(by=['f1-score'], ascending=False).head(5)
sato_top_5_df.index.name = 'type'
sato_bottom_5_df = report_sato_df.loc[report_sato_df['support']>0].sort_values(by=['f1-score'], ascending=False).tail(5)
sato_bottom_5_df.index.name = 'type'
sherlock_top_5_df = report_sherlock_df.loc[report_sherlock_df['support']>0].sort_values(by=['f1-score'], ascending=False).head(5)
sherlock_top_5_df.index.name = 'type'
sherlock_bottom_5_df = report_sherlock_df.loc[report_sherlock_df['support']>0].sort_values(by=['f1-score'], ascending=False).tail(5)
sherlock_bottom_5_df.index.name = 'type'


In [25]:
# altair chart representing the top 5 for both sato and sherlock
alt.Chart(sato_top_5_df.reset_index()).transform_fold(
        ['precision', 'recall', 'f1-score'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="f1-score",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [26]:
alt.Chart(sherlock_top_5_df.reset_index()).transform_fold(
        ['precision', 'recall', 'f1-score'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="f1-score",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [27]:
alt.Chart(sato_bottom_5_df.reset_index()).transform_fold(
        ['precision', 'recall', 'f1-score'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="f1-score",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [28]:
alt.Chart(sherlock_bottom_5_df.reset_index()).transform_fold(
        ['precision', 'recall', 'f1-score'],
      as_=['column', 'value']
    ).mark_bar(size=15).encode(
    column=alt.Column('type:O', sort=alt.EncodingSortField(
                field="f1-score",  
                order="descending")),
    x = alt.X('column:N',
              title = '',
              ),
    y=alt.Y('value:Q'),
    color='column:N'
)

alt.Chart(...)

In [29]:
path = '../../../gittables_benchmark/non_reannotated'

In [30]:
filepaths = [join(path, f) for f in listdir(path) if f.endswith('.csv')]
print(len(filepaths))
tables_of_interest = {}
single_col_tables = {}
col_idx = 0
multi_col_sherlock_increase = {}
sato_increase_count = 0
multi_col_count = 0
for idx, fp in enumerate(filepaths):
    table_id = fp[fp.rfind('/')+1:-4]
    table_header = pd.read_csv(fp,header=None, nrows=1).values[0][1:]
    table_df = pd.read_csv(fp, index_col=0)
    
    table_predictions_sato = []
    table_predictions_sherlock = []
    table_true_types = list(map(lambda x: x.split('.')[0], table_header))

    for idx, col_type in enumerate(table_header):
      table_predictions_sato.append(prediction_sato[col_idx])
      table_predictions_sherlock.append(prediction_sherlock[col_idx])
      col_idx += 1
    
    #check count of matches between table_predictions_sato and table_true_types
    #check count of matches between table_predictions_sherlock and table_true_types
    table_predictions_sato_count = 0
    table_predictions_sherlock_count = 0
    for idx, col_true_type in enumerate(table_true_types):
      if col_true_type == table_predictions_sato[idx]:
        table_predictions_sato_count += 1
      if col_true_type == table_predictions_sherlock[idx]:
        table_predictions_sherlock_count += 1
    
    if len(table_header) == 1:
        single_col_tables[table_id] = {
            'true_types': table_true_types,
            'predictions_sato': table_predictions_sato,
            'predictions_sherlock': table_predictions_sherlock,
            'predictions_sato_count': table_predictions_sato_count,
            'predictions_sherlock_count': table_predictions_sherlock_count,
            'table_df': table_df
        }
        continue
    else: multi_col_count += 1
        
    # Are there any multi-column tables where sherlock gets more correct predictions?
    if table_predictions_sato_count == table_predictions_sherlock_count: continue
    elif table_predictions_sato_count < table_predictions_sherlock_count: 
        multi_col_sherlock_increase[table_id] = {
            'true_types': table_true_types,
            'predictions_sato': table_predictions_sato,
            'predictions_sherlock': table_predictions_sherlock,
            'predictions_sato_count': table_predictions_sato_count,
            'predictions_sherlock_count': table_predictions_sherlock_count,
            'table_df': table_df
        }
    else: 
        sato_increase_count+= 1
        
    # match_freq_sato = table_predictions_sato_count / len(table_predictions_sato)
    # match_freq_sherlock = table_predictions_sherlock_count / len(table_predictions_sherlock)
    
    #if sherlock did not get any correct predictions, check if sato predicted at least 70% of the columns correct
    # otherwise check if the match count increased by 90% in sato compared to sherlock
    if (table_predictions_sherlock_count == 0 and table_predictions_sato_count==0): continue
    if ((table_predictions_sato_count/len(table_true_types) > 70/100) or (table_predictions_sherlock_count != 0 and ((table_predictions_sato_count-table_predictions_sherlock_count)/table_predictions_sherlock_count)*100 > 70/100)):
      tables_of_interest[table_id] = {
        'true_types': table_true_types,
        'predictions_sato': table_predictions_sato,
        'predictions_sherlock': table_predictions_sherlock,
        'predictions_sato_count': table_predictions_sato_count,
        'predictions_sherlock_count': table_predictions_sherlock_count,
        'table_df': table_df
      }
print(multi_col_count)

477
178


In [31]:
print(len(tables_of_interest))

67


In [32]:
print(sato_increase_count)
print(len(multi_col_sherlock_increase))
for k, v in multi_col_sherlock_increase.items():
    print(k)
    print(v)

95
20
GitTables_2637
{'true_types': ['name', 'rank', 'species', 'year'], 'predictions_sato': ['type', 'species', 'notes', 'year'], 'predictions_sherlock': ['species', 'rank', 'name', 'year'], 'predictions_sato_count': 1, 'predictions_sherlock_count': 2, 'table_df':           name        rank      species  year
0  Phalaroides       genus          NaN  1776
1  arundinacea     species          NaN  1969
2       caesia  subspecies  arundinacea  1973
3     japonica  subspecies  arundinacea  1973
4        picta     variety  arundinacea  1973
5       caesia     species          NaN  1977
6    hispanica     species          NaN  1977
7     japonica     species          NaN  1981
8        picta     species          NaN  2006
9     rotgesii     species          NaN  1977}
GitTables_1994
{'true_types': ['class', 'description'], 'predictions_sato': ['type', 'publisher'], 'predictions_sherlock': ['type', 'description'], 'predictions_sato_count': 0, 'predictions_sherlock_count': 1, 'table_df':      

In [33]:
for k, v in tables_of_interest.items():
    print(k)
    print(v)

GitTables_2004
{'true_types': ['name', 'rank', 'species', 'year'], 'predictions_sato': ['name', 'species', 'notes', 'year'], 'predictions_sherlock': ['species', 'species', 'address', 'year'], 'predictions_sato_count': 2, 'predictions_sherlock_count': 1, 'table_df':             name     rank  species    year
0   Theriophonum    genus      NaN  1837.0
1       crenatum  species      NaN  1837.0
2      dalzellii  species      NaN  1853.0
3       danielii  species      NaN  2010.0
4       fischeri  species      NaN  1981.0
5        indicum  species      NaN  1920.0
6      infaustum  species      NaN  1880.0
7        kleinii  species      NaN  1858.0
8     manickamii  species      NaN  2008.0
9        minutum  species      NaN  1895.0
10   chattarjeei  variety  minutum  1924.0
11  sivaganganum  species      NaN  1969.0
12   uniseriatum  species      NaN  1931.0
13       wightii  species      NaN  1879.0
14       wightii  species      NaN  1858.0
15    zeylanicum  species      NaN     NaN}
Gi

GitTables_2148
{'true_types': ['name', 'rank', 'species', 'year'], 'predictions_sato': ['name', 'species', 'notes', 'year'], 'predictions_sherlock': ['species', 'species', 'address', 'year'], 'predictions_sato_count': 2, 'predictions_sherlock_count': 1, 'table_df':                name     rank    species    year
0       Naemacyclus    genus        NaN  1873.0
1           alpinus  species        NaN  1875.0
2     arctostaphyli  species        NaN  1903.0
3             arxii  species        NaN  1972.0
4           caulium  species        NaN  1906.0
5        culmigenus  species        NaN  1894.0
6     durmitorensis  species        NaN  1915.0
7      fimbriatulus  species        NaN     NaN
8        fimbriatus  species        NaN  1983.0
9            flavus  species        NaN  1888.0
10        foliicola  species        NaN  1960.0
11           gilvus  species        NaN  1918.0
12          griseus  species        NaN  1889.0
13     hysterioides  species        NaN  1888.0
14           k

{'true_types': ['name', 'rank', 'year'], 'predictions_sato': ['name', 'species', 'year'], 'predictions_sherlock': ['address', 'species', 'year'], 'predictions_sato_count': 2, 'predictions_sherlock_count': 1, 'table_df':            name     rank  year
0  Spermaulaxen    genus  1836
1    dichotomus  species  1837
2      aphyllum  species  1824
3    phyllanthi  species  1806
4      strictum  species  1824}
GitTables_2831
{'true_types': ['name', 'rank', 'status', 'year'], 'predictions_sato': ['name', 'species', 'status', 'year'], 'predictions_sherlock': ['species', 'species', 'status', 'year'], 'predictions_sato_count': 3, 'predictions_sherlock_count': 2, 'table_df':            name     rank     status  year
0    Luttrellia    genus  conserved  1978
1     estuarina  species        NaN  1978
2     guttulata  species        NaN  2007
3      halonata  species        NaN  2007
4     monoceras  species    invalid  1978
5        oryzae  species    invalid  1978
6  parvulospora  species        Na

{'true_types': ['name', 'rank', 'year'], 'predictions_sato': ['name', 'species', 'year'], 'predictions_sherlock': ['species', 'species', 'year'], 'predictions_sato_count': 2, 'predictions_sherlock_count': 1, 'table_df':                 name     rank    year
0   Pseudotomentella    genus  1958.0
1             armata  species  2003.0
2         atrocyanea  species  1974.0
3          atrofusca  species  1971.0
4             cyanea  species  1960.0
5        echinospora  species  1958.0
6        flavovirens  species  1958.0
7             fumosa  species  1983.0
8   griseopergamacea  species  1971.0
9       griseoveneta  species  1974.0
10          humicola  species  1968.0
11      kaniksuensis  species  1983.0
12          larsenii  species  2001.0
13   longisterigmata  species     NaN
14          molybdea  species  1983.0
15          mucidula  species  1958.0
16         mucidulus  species     NaN
17             nigra  species  1958.0
18          ochracea  species  1998.0
19     rhizopunctata

In [34]:
sherlock_better = 0
sato_better = 0
equal_score = 0
for k, v in single_col_tables.items():
#     print(k)
#     print(v)
    if v['predictions_sato_count'] == v['predictions_sherlock_count']: equal_score += 1
    elif v['predictions_sato_count'] < v['predictions_sherlock_count']: sherlock_better += 1
    else: sato_better += 1
print(sherlock_better/(sherlock_better+sato_better+equal_score))
print(sato_better/(sherlock_better+sato_better+equal_score))
print(equal_score/(sherlock_better+sato_better+equal_score))

0.24414715719063546
0.1605351170568562
0.5953177257525084
